In [1]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
    format='%(asctime)s - %(levelname)-8s | %(name)-20s | %(message)s',
    level=logging.INFO
    )

In [2]:
import time

from static import TEAMS_UNDERSTAT, YEARS
from understat import get_soup

In [3]:
dfs = []
for year in YEARS:
    for team in TEAMS_UNDERSTAT:
        logger.info(f"Scraping data from Year {year} and Team {team}")
        
        df = get_soup(team=team, year=year)

        if df is None:
            logger.info(f"DATA NOT FOUND")
            continue
        
        df["Year"] = int(year)
        
        dfs.append(df)
        time.sleep(2.5)

2024-08-19 18:51:20,675 - INFO     | __main__             | Scraping data from Year 2024 and Team Inter
2024-08-19 18:51:23,476 - INFO     | __main__             | Scraping data from Year 2024 and Team AC_Milan
2024-08-19 18:51:26,284 - INFO     | __main__             | Scraping data from Year 2024 and Team Juventus
2024-08-19 18:51:29,181 - INFO     | __main__             | Scraping data from Year 2024 and Team Atalanta
2024-08-19 18:51:31,973 - INFO     | __main__             | Scraping data from Year 2024 and Team Bologna
2024-08-19 18:51:34,884 - INFO     | __main__             | Scraping data from Year 2024 and Team Roma
2024-08-19 18:51:37,678 - INFO     | __main__             | Scraping data from Year 2024 and Team Lazio
2024-08-19 18:51:40,604 - INFO     | __main__             | Scraping data from Year 2024 and Team Fiorentina
2024-08-19 18:51:43,524 - INFO     | __main__             | Scraping data from Year 2024 and Team Napoli
2024-08-19 18:51:46,428 - INFO     | __main__   

In [2]:
import pandas as pd

In [10]:
df = pd.concat(dfs, ignore_index=True)

In [15]:
df.to_csv("../db/raw/understat.csv", index=False)

In [7]:
def extract_last_name(full_name):
    name_parts = full_name.split()
    if len(name_parts) > 2:
        return ' '.join(name_parts[-2:])  # Last two parts are the last name
    else:
        return name_parts[-1]  # Last part is the last name

In [28]:
understat = pd.read_csv("../db/raw/understat.csv").query("Year == 2023")
fleghe = pd.read_csv("../db/clean/stats.csv").query("Year == 2024")

In [29]:
understat['last_name'] = understat['player_name'].apply(extract_last_name)
understat['first_initial'] = understat['player_name'].apply(lambda x: x.split()[0][0])

# Step 2: Identify duplicates
duplicate_last_names = understat['last_name'].duplicated(keep=False)

# Step 3: Append the first initial to duplicates only
understat.loc[duplicate_last_names, 'merge_key'] = understat['last_name'] + ' ' + understat['first_initial'] + '.'
understat.loc[~duplicate_last_names, 'merge_key'] = understat['last_name']

In [30]:
understat.drop(columns=['id', 'Year', 'player_name'], inplace=True)

In [31]:
understat.head()

,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup,last_name,first_initial,merge_key
614,33,2688,24,19.917104,3,4.228376,109,36,5,0,F S,Inter,22,17.633207,29.067482,11.011354,Martínez,L,Martínez L.
615,32,2618,13,9.977345,3,4.402720,62,51,5,0,M,Inter,3,2.364357,21.514876,18.073472,Calhanoglu,H,Calhanoglu
616,35,2750,13,15.489109,7,4.381468,77,30,3,0,F S,Inter,13,15.489109,25.697543,8.535634,Thuram,M,Thuram
617,32,877,6,8.650839,3,2.203673,32,10,1,0,M S,Inter,6,8.650839,12.410261,3.090515,Frattesi,D,Frattesi
618,27,748,5,5.603071,3,2.895867,17,16,0,0,F S,Inter,5,5.603071,9.664403,2.593221,Arnautovic,M,Arnautovic


In [32]:
merged_df = pd.merge(fleghe, understat, left_on='Name', right_on='merge_key', how='left')

In [39]:
merged_df.to_csv("../db/clean/stats_merged.csv", index=False)

In [43]:
merged_df.columns

Index(['Id', 'R', 'Name', 'Team', 'Year', 'Pv', 'Mv', 'Fm', 'games', 'time',
       'goals', 'xG', 'assists', 'xA', 'shots', 'key_passes', 'yellow_cards',
       'red_cards', 'position', 'team_title', 'npg', 'npxG', 'xGChain',
       'xGBuildup', 'last_name', 'first_initial', 'merge_key'],
      dtype='object')